In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager  
from apscheduler.schedulers.background import BackgroundScheduler
from dotenv import load_dotenv
import traceback

# 환경 변수 로드
load_dotenv()

# 환경 변수에서 아이디와 비밀번호 가져오기
USER_ID = os.getenv('USER_ID')
USER_PASSWORD = os.getenv('USER_PASSWORD')

# 기존 CSV 파일 경로
CSV_FILE_PATH = '2024_vacation.csv'

# 크롤링할 URL 리스트
URLS = [
    'https://ntoday.daouoffice.com/app/approval/manage/document?startAt=2024-01-01T00%3A00%3A00.000%2B09%3A00&endAt=2024-12-30T23%3A59%3A59.000%2B09%3A00&drafterName=&drafterDeptName=&activityUserName=&docNum=&title=&docStatus%5B%5D=inprogress&docStatus%5B%5D=complete&docStatus%5B%5D=return&docStatus%5B%5D=recv_waiting&docStatus%5B%5D=received&docStatus%5B%5D=recv_returned&docType%5B%5D=draft&docType%5B%5D=receive&formId%5B%5D=206986&page=0&offset=20&property=id&direction=desc&integration%5B%5D=nonUse&integration%5B%5D=use&docId=&apprStatus=',
    'https://ntoday.daouoffice.com/app/approval/manage/document?startAt=2024-01-01T00%3A00%3A00.000%2B09%3A00&endAt=2024-12-31T23%3A59%3A59.000%2B09%3A00&drafterName=&drafterDeptName=&activityUserName=&docNum=&title=&docStatus%5B%5D=inprogress&docStatus%5B%5D=complete&docStatus%5B%5D=return&docStatus%5B%5D=recv_waiting&docStatus%5B%5D=received&docStatus%5B%5D=recv_returned&docType%5B%5D=draft&docType%5B%5D=receive&formId%5B%5D=256338&page=0&offset=20&property=id&direction=desc&integration%5B%5D=nonUse&integration%5B%5D=use&docId=&apprStatus='
]

# 사용자 정의 예외 클래스 생성 (중복 수집 방지)
class StopCrawlingException(Exception):
    pass

# CSV 파일 로드 함수
def load_existing_data(file_path):
    if os.path.exists(file_path):
        try:
            data = pd.read_csv(
                file_path,
                dtype={'문서 번호': str},
            )
            print(f"기존 데이터 로드 완료: {len(data)}건")
            return data
        except Exception as e:
            print(f"데이터 로드 중 오류 발생: {e}")
            traceback.print_exc()
            return pd.DataFrame(columns=[
                '문서 번호', '기안자 이름', '기안 부서', '기안일', '휴가 종류',
                '시작 날짜', '종료 날짜', '잔여 포인트', '신청 포인트', '휴가 사유', '승인 여부'
            ])
    else:
        print("기존 CSV 파일이 존재하지 않아 새로 생성합니다.")
        return pd.DataFrame(columns=[
            '문서 번호', '기안자 이름', '기안 부서', '기안일', '휴가 종류',
            '시작 날짜', '종료 날짜', '잔여 포인트', '신청 포인트', '휴가 사유', '승인 여부'
        ])

# 데이터 업데이트 함수
def update_csv_file(file_path, new_data):
    try:
        existing_data = load_existing_data(file_path)
        combined_data = pd.concat([existing_data, new_data], ignore_index=True)
        combined_data = combined_data.drop_duplicates(subset=['문서 번호'], keep='last')

        # 기안일 기준으로 정렬
        combined_data = combined_data.sort_values(by='기안일')

        # 날짜를 문자열 형식으로 저장
        combined_data.to_csv(file_path, index=False, encoding='utf-8-sig')
        print("데이터가 CSV 파일로 업데이트되었습니다.")
    except Exception as e:
        print(f"CSV 파일 업데이트 중 오류 발생: {e}")
        traceback.print_exc()

# 크롤링 수행 중 중복 체크 함수
def is_duplicate(doc_number, existing_doc_numbers_set):
    # 기존 데이터프레임에서 중복 여부 체크 (집합을 사용하여 빠르게 검사)
    is_dup = doc_number.strip() in existing_doc_numbers_set
    if is_dup:
        print(f"중복 확인됨: 문서 번호 {doc_number}")
    else:
        print(f"중복되지 않음: 문서 번호 {doc_number}")
    return is_dup

# 날짜 전처리 함수
def clean_date(date):
    if pd.isna(date):
        return ''
    # 요일 정보를 제거하고 날짜 형식으로 변환 (예: '2024-01-01(월)' -> '2024-01-01')
    date_cleaned = date.split('(')[0].strip()
    return date_cleaned


def crawl(urls):
    print("크롤링 시작")
    # 웹 드라이버 옵션 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    # options.add_argument('--headless')  # 필요 시 헤드리스 모드 활성화

    # 웹 드라이버 초기화
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

    try:
        # 첫 번째 URL로 접속하여 로그인
        driver.get(urls[0])
        print(f"첫 번째 URL로 접속: {urls[0]}")

        # 로그인 처리
        try:
            id_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/form/section/fieldset/div[1]/input'))
            )
            id_input.send_keys(USER_ID)  # 아이디 입력
            print("아이디 입력 완료")

            password_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/form/section/fieldset/div[2]/input'))
            )
            password_input.send_keys(USER_PASSWORD)  # 비밀번호 입력
            print("비밀번호 입력 완료")

            login_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/form/section/fieldset/a'))
            )
            login_button.click()
            print("로그인 버튼 클릭 완료")

            # 페이지가 로드된 후 스킵 버튼 클릭
            skip_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div[5]/a[1]'))
            )
            skip_button.click()
            print("스킵 버튼 클릭 완료")
        except Exception as e:
            print(f"로그인 중 오류 발생: {e}")
            traceback.print_exc()
            driver.quit()
            return

        # 기존 데이터 불러오기
        existing_data = load_existing_data(CSV_FILE_PATH)
        existing_doc_numbers_set = set(existing_data['문서 번호'].str.strip())

        # 새로운 데이터를 저장하는 리스트
        data_list = []

        # 각 URL에 대해 크롤링 수행
        for current_url in urls:
            print(f"현재 크롤링 중인 URL: {current_url}")
            try:
                driver.get(current_url)
                print(f"URL 접속 완료: {current_url}")
                time.sleep(3)  # 페이지 로딩 대기

                while True:
                    # tbody의 모든 tr 요소 가져오기
                    rows = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, '//table[@class="type_normal list_approval"]/tbody/tr'))
                    )
                    num_rows = len(rows)
                    print(f"현재 페이지의 문서 수: {num_rows}")

                    # 각 문서를 순회하면서 데이터 추출
                    for row_index in range(1, num_rows + 1):
                        try:
                            # 문서 번호 추출
                            doc_number_xpath = f'//table[@class="type_normal list_approval"]/tbody/tr[{row_index}]/td[2]/span'
                            doc_number_element = WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.XPATH, doc_number_xpath))
                            )
                            doc_number = doc_number_element.text.strip()
                            print(f"문서 번호 추출 완료: {doc_number}")

                            # 중복 체크
                            if is_duplicate(doc_number, existing_doc_numbers_set):
                                print(f"문서 번호 {doc_number}는 이미 처리되었습니다. 다음 문서로 이동합니다.")
                                continue  # 중복된 경우, 현재 문서를 건너뛰고 다음 문서로 이동

                            # 완료 여부 추출: 완료 또는 반려 상태 확인
                            status_xpath = f'/html/body/div[1]/div[2]/div[2]/div/div[2]/table/tbody/tr[{row_index}]/td[11]/a/span'
                            status_element = WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.XPATH, status_xpath))
                            )
                            status = status_element.text.strip()  # 완료 여부 추출 (예: "완료" 또는 "반려")
                            
                            # 문서 링크 클릭
                            link_xpath = f'//table[@class="type_normal list_approval"]/tbody/tr[{row_index}]/td[6]/a/span'
                            link_element = WebDriverWait(driver, 10).until(
                                EC.element_to_be_clickable((By.XPATH, link_xpath))
                            )
                            link_element.click()
                            print(f"{row_index}번째 문서 클릭 완료")
                            time.sleep(2) 


                            # 데이터 추출
                            drafter_name = WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, 'span[data-id="draftUser"]'))
                            ).text
                            drafter_dept = driver.find_element(By.CSS_SELECTOR, 'span[data-id="draftDept"]').text
                            draft_date = driver.find_element(By.CSS_SELECTOR, 'span[data-id="draftDate"]').text
                            vacation_type = driver.find_element(By.CSS_SELECTOR, 'span[data-name="select_type"]').text
                            start_date = driver.find_element(By.CSS_SELECTOR, 'span[data-id="startDate"]').text
                            end_date = driver.find_element(By.CSS_SELECTOR, 'span[data-id="endDate"]').text
                            rest_point = driver.find_element(By.CSS_SELECTOR, 'span[data-id="restPoint"]').text
                            apply_point = driver.find_element(By.CSS_SELECTOR, 'span[data-id="applyPoint"]').text
                            description = driver.find_element(By.CSS_SELECTOR, 'span[data-id="description"]').text

                            # 날짜 전처리
                            start_date_cleaned = clean_date(start_date)
                            end_date_cleaned = clean_date(end_date)
                            draft_date_cleaned = clean_date(draft_date)

                            # 추출한 데이터를 리스트에 저장
                            data_list.append([
                                doc_number, drafter_name, drafter_dept, draft_date_cleaned, vacation_type,
                                start_date_cleaned, end_date_cleaned, rest_point, apply_point, description, status
                            ])
                            print(f"데이터 수집 완료: 문서 번호 {doc_number}")

                            # 뒤로 가기
                            driver.back()
                            WebDriverWait(driver, 20).until(
                                EC.presence_of_all_elements_located((By.XPATH, '//table[@class="type_normal list_approval"]/tbody/tr'))
                            )
                            time.sleep(1)

                        except (StaleElementReferenceException, TimeoutException) as e:
                            print(f"{row_index}번째 문서 접근 중 오류 발생: {e}. 문서를 건너뜁니다.")
                            traceback.print_exc()
                            try:
                                driver.back()
                                WebDriverWait(driver, 10).until(
                                    EC.presence_of_all_elements_located((By.XPATH, '//table[@class="type_normal list_approval"]/tbody/tr'))
                                )
                                time.sleep(1)
                            except Exception as back_error:
                                print(f"오류 후 뒤로 가기 중 추가 오류 발생: {back_error}")
                            continue  # 다음 문서로 이동

                        except Exception as e:
                            print(f"{row_index}번째 문서 접근 중 예상치 못한 오류 발생: {e}. 문서를 건너뜁니다.")
                            traceback.print_exc()
                            try:
                                driver.back()
                                WebDriverWait(driver, 10).until(
                                    EC.presence_of_all_elements_located((By.XPATH, '//table[@class="type_normal list_approval"]/tbody/tr'))
                                )
                                time.sleep(1)
                            except Exception as back_error:
                                print(f"오류 후 뒤로 가기 중 추가 오류 발생: {back_error}")
                            continue  # 다음 문서로 이동

                    # 페이지 끝에 도달했는지 확인 후 종료
                    try:
                        current_page = driver.find_element(By.CSS_SELECTOR, 'a.paginate_button.paginate_active').text
                        print(f"현재 페이지: {current_page}")

                        next_button = WebDriverWait(driver, 20).until(
                            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/a[3]'))
                        )
                        next_button.click()
                        WebDriverWait(driver, 10).until(
                            EC.presence_of_all_elements_located((By.XPATH, '//table[@class="type_normal list_approval"]/tbody/tr'))
                        )
                        time.sleep(2)

                        # 다음 페이지 번호 확인
                        new_page = driver.find_element(By.CSS_SELECTOR, 'a.paginate_button.paginate_active').text
                        print(f"새로운 페이지: {new_page}")

                        if current_page == new_page:
                            print("다음 버튼을 클릭해도 페이지가 변경되지 않았습니다. 수집을 종료합니다.")
                            break

                    except TimeoutException as page_error:
                        print(f"다음 페이지로 이동 중 TimeoutException 발생: {page_error}")
                        break  # 다음 페이지 이동 중 오류 발생 시, 해당 URL의 크롤링 종료

                # 크롤링이 종료된 후 수집된 데이터 업데이트
                if data_list:
                    try:
                        # 새로운 데이터를 데이터프레임으로 변환
                        new_data_df = pd.DataFrame(data_list, columns=[
                            '문서 번호', '기안자 이름', '기안 부서', '기안일', '휴가 종류',
                            '시작 날짜', '종료 날짜', '잔여 포인트', '신청 포인트', '휴가 사유', '승인 여부'
                        ])

                        # CSV 파일 업데이트
                        update_csv_file(CSV_FILE_PATH, new_data_df)
                    except Exception as e:
                        print(f"데이터프레임 변환 또는 CSV 파일 업데이트 중 오류 발생: {e}")
                        traceback.print_exc()

            except Exception as e:
                print(f"{current_url} URL 크롤링 중 예상치 못한 오류 발생: {e}")
                traceback.print_exc()
                continue  # 다음 URL로 이동


    finally:
        driver.quit()
        print("웹 드라이버 종료")

# 스케줄링 함수
def schedule_crawling():
    # 크롤링할 URL 리스트
    urls = URLS  # 필요한 경우 추가적인 URL을 리스트에 포함시키세요.
    crawl(urls)

if __name__ == "__main__":
    # APScheduler 설정
    scheduler = BackgroundScheduler()
    
    # 특정 주기 설정: 예를 들어, 매일 오전 2시에 크롤링 수행
    scheduler.add_job(schedule_crawling, 'interval', minutes=5, next_run_time=pd.Timestamp.now())
    
    # 스케줄러 시작
    scheduler.start()
    print("스케줄러가 시작되었습니다. 지정된 주기에 따라 크롤링이 자동으로 수행됩니다.")
    
    try:
        # 메인 스레드를 계속 실행 상태로 유지
        while True:
            time.sleep(1)
    except (KeyboardInterrupt, SystemExit):
        # 프로그램 종료 시 스케줄러 종료
        scheduler.shutdown()
        print("스케줄러가 종료되었습니다.")

스케줄러가 시작되었습니다. 지정된 주기에 따라 크롤링이 자동으로 수행됩니다.크롤링 시작

첫 번째 URL로 접속: https://ntoday.daouoffice.com/app/approval/manage/document?startAt=2024-01-01T00%3A00%3A00.000%2B09%3A00&endAt=2024-12-30T23%3A59%3A59.000%2B09%3A00&drafterName=&drafterDeptName=&activityUserName=&docNum=&title=&docStatus%5B%5D=inprogress&docStatus%5B%5D=complete&docStatus%5B%5D=return&docStatus%5B%5D=recv_waiting&docStatus%5B%5D=received&docStatus%5B%5D=recv_returned&docType%5B%5D=draft&docType%5B%5D=receive&formId%5B%5D=206986&page=0&offset=20&property=id&direction=desc&integration%5B%5D=nonUse&integration%5B%5D=use&docId=&apprStatus=
아이디 입력 완료
비밀번호 입력 완료
로그인 버튼 클릭 완료
스킵 버튼 클릭 완료
기존 데이터 로드 완료: 374건
현재 크롤링 중인 URL: https://ntoday.daouoffice.com/app/approval/manage/document?startAt=2024-01-01T00%3A00%3A00.000%2B09%3A00&endAt=2024-12-30T23%3A59%3A59.000%2B09%3A00&drafterName=&drafterDeptName=&activityUserName=&docNum=&title=&docStatus%5B%5D=inprogress&docStatus%5B%5D=complete&docStatus%5B%5D=return&docStatus%5B%5D=recv_waiti